In [1]:
import dataset
import pandas as pd
import json
import time
import secrets
import datetime
from datetime import timedelta
import requests
import urllib.parse
import sqlite3

# import the class that ingests a 1% sampling of all tweets via the Twitter API 
from Sample_TwitterListener_FINAL import Sample

# import the class that ingests tweets based on the provided keyword(s)
from Filter_TwitterListener_FINAL import Filter

In [2]:
# settings for viewing data frames so we can see all rows and columns 

# set view parameters for the pandas data frames 
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999


In [3]:
# retrieve secrets 
consumerKey = secrets.consumerKey
consumerSecret = secrets.consumerSecret
accessToken = secrets.accessToken
accessTokenSecret = secrets.accessTokenSecret

In [4]:
%%time

# code to use the listener to retrieve a random sampling of tweets 
# it will pull in ~1% of all tweets 
# language=en parameter is used to only pull in English language tweets 
# the inputted value will limit the number of tweets retrieved, otherwise the script would run indefinitely or 
# until it enough random connection issues occured that would lead to the script disconnecting. 


tweet_count = 0
tweet_count = int(input ("How large do you want your tweet sample size to be? "))
scraper = Sample(consumerKey, consumerSecret, accessToken, accessTokenSecret, tweet_count)
scraper.sample(languages=['en'])



How large do you want your tweet sample size to be? 10
Good project
https://t.co/N2ekBaCk3A


Stream connection closed by Twitter


stream disconnecting, desired sample size received
CPU times: user 728 ms, sys: 41.1 ms, total: 770 ms
Wall time: 5.65 s


In [5]:
%%time 

# quick SQL query to verify that the data landed in the SQLite DB properly 

# query DB being populated by Twitter listener 
con = sqlite3.connect("sampletweetsdb.db")

# select the entire table 
sample_df = pd.read_sql_query("SELECT * from sample_tweet_table_two", con)

# close the connection 
con.close()

# display the data 
sample_df.head(10)




CPU times: user 4.06 ms, sys: 1.73 ms, total: 5.79 ms
Wall time: 4.5 ms


,id,text,timeStamp,verified_user,followers,retweetCount,favoriteCount,location,textblob_sentiment,text_blob_subjectivity,vader_compound,vader_positive,vader_negative,vader_neutral
0,1,@emily_tweets Enjoy every moment,2022-04-24 03:05:29.000000,0,137,0,0,None,0.400000,0.500,0.4939,0.516,0.000,0.484
1,2,Pat Bev better watch his step. You don't provo...,2022-04-24 03:05:29.000000,1,51509,0,0,None,0.500000,0.500,0.6319,0.340,0.000,0.660
2,3,"I need a pedi, full set, lashes, brows and hai...",2022-04-24 03:05:29.000000,0,85,0,0,None,0.350000,0.550,0.0000,0.000,0.000,1.000
3,4,Cutest,2022-04-24 03:05:29.000000,0,658,0,0,None,0.000000,0.000,0.5859,1.000,0.000,0.000
4,5,#معّصيِتيُ_رَاحتْي **** https://t.co/hFsYDVwswc,2022-04-24 03:05:29.000000,0,68,0,0,None,0.000000,0.000,0.0000,0.000,0.000,1.000
5,6,@Shill_Ronin morning,2022-04-24 03:05:30.000000,0,570,0,0,None,0.000000,0.000,0.0000,0.000,0.000,1.000
6,7,I thought he decked him lmaoo,2022-04-24 03:05:30.000000,0,203,0,0,None,0.000000,0.000,0.0000,0.000,0.000,1.000
7,8,No he's 42 silly 🤦🏼‍♂️🤣,2022-04-24 03:05:30.000000,0,939,0,0,None,-0.500000,0.875,-0.2732,0.175,0.349,0.476
8,9,@topstarnews @forever_951013 So excited to hea...,2022-04-24 03:05:30.000000,0,20,0,0,None,0.375000,0.750,0.4005,0.071,0.000,0.929
9,10,So I decided to open the mx518 and use the usb...,2022-04-24 03:05:30.000000,0,24,0,0,None,-0.133333,0.400,-0.4019,0.000,0.053,0.947


In [6]:
%%time

# This section will pull in a real time filtered stream based on the inputted search terms 

try: 
    tweet_count = 0
    tweet_count = int(input ("How large do you want your tweet sample size to be? "))
    count = 0
    phrase_list = []
    phrases_count = int(input ("Please enter how many terms or phrases to filter by: "))
    while (count < phrases_count):
        term = input("Please enter keyword or hastag to search:" )
        phrase_list.append(term)
        count+=1

    scraper = Filter(consumerKey, consumerSecret, accessToken, accessTokenSecret, tweet_count, phrase_list)
    scraper.filter(track=phrase_list, languages=['en'])
    
except KeyboardInterrupt:
    print("Stopped.")

How large do you want your tweet sample size to be? 10
Please enter how many terms or phrases to filter by: 2
Please enter keyword or hastag to search:Taco Bell
Please enter keyword or hastag to search:Chipotle
I used to joke at the Taco Bell by my high school where everyone would go get **** in the parking lot on Friday nights (except me because I was weird) that it’s impossible to spend $8 at Taco Bell. No one could eat that much food.

Minimum wage was also $4.25 an hour tho, so…


Stream connection closed by Twitter


stream disconnecting, desired sample size received
CPU times: user 896 ms, sys: 93.3 ms, total: 989 ms
Wall time: 3min 27s


In [7]:
%%time 

#quick look at the DB To see how many records there are

# query DB being populated by Twitter listener 
con = sqlite3.connect("filtertweetsdb.db")

# select the entire table 
filter_df = pd.read_sql_query("SELECT * from filter_tweet_table", con)

# close the connection 
con.close()

# display the data 
filter_df.head(10)

CPU times: user 4 ms, sys: 8.05 ms, total: 12 ms
Wall time: 11.4 ms


,id,keyword,text,timeStamp,verified_user,followers,retweetCount,favoriteCount,location,textblob_sentiment,textblob_subjectivity,vader_compound,vader_positive,vader_negative,vader_neutral
0,1,McDonald's,@saandravalera McDonald's,2022-04-24 03:07:42.000000,0,98,0,0,None,0.000000,0.000000,0.0000,0.000,0.000,1.000
1,2,Wendy's,@pmarca 50 Dollars For #1 Wendy's Triple Mediu...,2022-04-24 03:08:19.000000,0,266,0,0,None,0.200000,0.350000,0.4740,0.089,0.025,0.886
2,3,McDonald's,@standard_mcd McDonald's is here: https://t.co...,2022-04-24 03:09:58.000000,0,6280,0,0,None,0.000000,0.000000,0.0000,0.000,0.000,1.000
3,4,Wendy's,Why the 𝑭𝑼𝑪𝑲 does Wendy's **** always look so ...,2022-04-24 03:09:59.000000,0,78,0,0,None,0.050000,0.750000,-0.5877,0.132,0.289,0.579
4,5,McDonald's,@bodybyroadkill @CryptoModeler @Tjdriii Y'all ...,2022-04-24 03:10:13.000000,0,263,0,0,None,0.119048,0.223810,0.6249,0.130,0.000,0.870
5,6,Taco Bell,"Cleaned my apartment, washed my clothes/sheets...",2022-04-24 03:20:11.000000,0,922,0,0,None,0.475000,0.466667,0.4404,0.097,0.000,0.903
6,7,Taco Bell,@JonSolo Taco Bell,2022-04-24 03:20:29.000000,0,4,0,0,None,0.000000,0.000000,0.0000,0.000,0.000,1.000
7,8,Taco Bell,@jon_bois Shout out to the time one of my frie...,2022-04-24 03:21:44.000000,0,768,0,0,None,0.000000,0.000000,0.7351,0.279,0.000,0.721
8,9,Taco Bell,@jacksantucci ****. Taco Bell crushes Wendy’s/...,2022-04-24 03:22:10.000000,0,1020,0,0,None,0.000000,0.000000,-0.7717,0.000,0.427,0.573
9,10,Taco Bell,Dude at Taco Bell was like man Ian give u no s...,2022-04-24 03:22:36.000000,0,1532,0,0,None,0.000000,0.000000,0.0772,0.134,0.118,0.749
